In [45]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import pandas as pd
import time

GOOGLE'S GEOCODING API:
https://developers.google.com/maps/documentation/geocoding

In [37]:
load_dotenv()
#This loads The built-in Python OS libary which speaks to your operating system
#And in this case we use it to retrieve a hidden environmental variable
import os
api_key = os.environ.get('GMAP_API')

### Using Google maps API for authors


In [48]:
df = pd.read_csv('author-address.csv')
df


,Unnamed: 0,gr_link,author_place
0,0,https://www.goodreads.com/author/show/3780.Jam...,"Newburgh, New York, The United States"
1,1,https://www.goodreads.com/author/show/706255.S...,"Skelleftehamn , Sweden"
2,4,https://www.goodreads.com/author/show/24556.Sa...,"Vancouver, Canada"
3,5,https://www.goodreads.com/author/show/721.John...,"Jonesboro, Arkansas, The United States"
4,6,https://www.goodreads.com/author/show/93353.Ab...,"Adis Ababa, Ethiopia"
...,...,...,...
635,2616,https://www.goodreads.com/author/show/14584166...,The United States
636,2618,https://www.goodreads.com/author/show/150038.C...,"Tehran, Iran"
637,2620,https://www.goodreads.com/author/show/21163293...,The United States
638,2628,https://www.goodreads.com/author/show/22182916...,The United States


In [50]:
geometry_df = []
for index, row in df.iterrows(): 
    place = row['author_place']
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json'
        keys = {'address': place, 'key': api_key}
        r = requests.get(url,params=keys)
        result_dic = r.json()
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = []
        coords.append(long)
        coords.append(lat)
        mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords}
        geometry_df.append(mygeometry)
        print(mygeometry)
        time.sleep(1)
    except Exception as e:
        print("Failed with", e)
        # If it fails, continue to the next entry
        continue

{'place': 'Newburgh, New York, The United States', 'geometry.type': 'Point', 'geometry.coordinates': [-74.0104178, 41.5034271]}
{'place': 'Skelleftehamn , Sweden', 'geometry.type': 'Point', 'geometry.coordinates': [21.2361263, 64.6861026]}
{'place': 'Vancouver, Canada', 'geometry.type': 'Point', 'geometry.coordinates': [-123.1207375, 49.2827291]}
{'place': 'Jonesboro, Arkansas, The United States', 'geometry.type': 'Point', 'geometry.coordinates': [-90.7051346, 35.835696]}
{'place': 'Adis Ababa, Ethiopia', 'geometry.type': 'Point', 'geometry.coordinates': [38.7524635, 9.0191936]}
{'place': 'Olean, The United States', 'geometry.type': 'Point', 'geometry.coordinates': [-78.42992699999999, 42.083639]}
{'place': 'Jackson, Mississippi, The United States', 'geometry.type': 'Point', 'geometry.coordinates': [-90.1848103, 32.2987573]}
{'place': 'Brooklyn, New York, The United States', 'geometry.type': 'Point', 'geometry.coordinates': [-73.9441579, 40.6781784]}
{'place': 'Manchester, Georgia, The

In [51]:
#Writing my final geometry data frame to a JSON file
#On my computer!!!
import json
with open('result_df12-9.json', 'w') as fp:
    json.dump(geometry_df, fp)

In [66]:
df_places = pd.DataFrame(geometry_df)
df_places.drop_duplicates(subset=['place'], inplace=True)

In [67]:
df_places

,place,geometry.type,geometry.coordinates
0,"Newburgh, New York, The United States",Point,"[-74.0104178, 41.5034271]"
1,"Skelleftehamn , Sweden",Point,"[21.2361263, 64.6861026]"
2,"Vancouver, Canada",Point,"[-123.1207375, 49.2827291]"
3,"Jonesboro, Arkansas, The United States",Point,"[-90.7051346, 35.835696]"
4,"Adis Ababa, Ethiopia",Point,"[38.7524635, 9.0191936]"
...,...,...,...
502,"Martinsburg, The United States",Point,"[-77.96388689999999, 39.4562099]"
504,"Charleston, SC, The United States",Point,"[-79.93105120000001, 32.7764749]"
506,"Arizona, The United States",Point,"[-111.0937311, 34.0489281]"
508,"Umeå, Sweden",Point,"[20.2630354, 63.8258471]"


In [68]:
# merge back to original df

df_add_merged = df.merge(df_places, left_on='author_place', right_on='place', how='left')

In [69]:
df_add_merged.to_csv('gr_author_address.csv')